In [ ]:
!pip install datasets transformers huggingface_hub
!pip install accelerate -U
!apt-get install git-lfs

import torch
import pandas as pd
import transformers
from datasets import Dataset
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer


torch.cuda.is_available()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


True

In [ ]:
path = "/content/IMDB Dataset.csv"
df = pd.read_csv(path)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.dropna(inplace = True)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [ ]:
def preprocess_text(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    review = soup.get_text()
    review = re.sub(r'\[[^]]*\]', ' ', review)
    review = re.sub(r'[^a-zA-Z]', ' ', review)
    review = review.split()
    return ' '.join(review)

In [ ]:
df['review'] = df['review'].apply(preprocess_text)

<ipython-input-9-b4d15d81e83b>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_text, "html.parser")


In [ ]:
df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0})

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

## Assuming 'target' is the column you want to stratify based on
target_column = 'sentiment'

# Create a StratifiedShuff Create a StratifiedShuffleSplit object for the initial split (10%)
initial_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.8, random_state=42)

# Get the indices for the initial stratified subset
initial_train_index, initial_test_index = next(initial_splitter.split(df, df[target_column]))

# Create the initial training and testing sets
initial_train_df, initial_test_df = df.iloc[initial_train_index], df.iloc[initial_test_index]

# Further split the initial training set into training and testing subsets (20%)
train_df, test_df = train_test_split(
    initial_train_df,
    test_size=0.2,
    stratify=initial_train_df[target_column],
    random_state=42
)
#len(final_test_df)

In [ ]:
#train_df, test_df = train_test_split(df, test_size=0.2)  # Adjust the test_size as needed

train_data_dict = {
    "text": train_df["review"].tolist(),
    "label": train_df["sentiment"].tolist(),
}

test_data_dict = {
    "text": test_df["review"].tolist(),
    "label": test_df["sentiment"].tolist(),
}

train_dataset = Dataset.from_dict(train_data_dict)
test_dataset = Dataset.from_dict(test_data_dict)

In [ ]:
model_name = "bert-base-uncased"  # You can choose a different BERT model as well
tokenizer = BertTokenizer.from_pretrained(model_name)
#model = BertForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation=True)




In [ ]:
# Define a custom model class for training from scratch
class BertForSequenceClassificationFromScratch(BertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.init_weights()

# Initialize the custom model without loading pre-trained weights
num_labels = 2  # Binary classification
config = BertForSequenceClassification.from_pretrained(model_name).config
config.num_labels = num_labels
model = BertForSequenceClassificationFromScratch(config)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
from transformers import TrainingArguments, Trainer
from accelerate import Accelerator
from transformers import TrainerCallback
from copy import deepcopy

training_args = TrainingArguments(
    per_device_train_batch_size=8,
    output_dir="./sentiment_finetuned_model",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=2,
    num_train_epochs=10,
)

# Define an Accelerator to enable mixed-precision training and distributed training
accelerator = Accelerator()


In [ ]:
class CustomCallback(TrainerCallback):

    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer

    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy


In [ ]:
# Define a Trainer using the TrainingArguments and Accelerator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)
trainer.add_callback(CustomCallback(trainer))
# Train the model for 10 epochs and collect accuracy information
train_acc_list = []
eval_acc_list = []


trainresults=trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(trainresults)
print(eval_results)
# Collect accuracy information
#train_acc_list.append(eval_results["train_accuracy"])
#eval_acc_list.append(eval_results["eval_accuracy"])

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.719600,0.704828,0.500000,0.000000
2,0.710200,0.705533,0.500000,0.000000


<ipython-input-18-b571dbad15c2>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.719600,0.704828,0.500000,0.000000
2,0.710200,0.705533,0.500000,0.000000


In [ ]:

df=pd.DataFrame(trainer.state.log_history)


In [ ]:
df

In [ ]:
df.to_excel("imdbscratch20.xlsx", index=False)

In [ ]:
from google.colab import files
files.download("imdbscratch20.xlsx")